# Scraping Web Data and Data Cleaning

We use Beautiful Soup library to scrap web data 

In [63]:
# !conda install folium -y
# !pip install pycountry fiona geopandas

In [124]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pycountry

import requests
from bs4 import BeautifulSoup

## Scrap Country Population and Land Area data

In [75]:
url = 'https://www.worldometers.info/world-population/population-by-country/'
response = requests.get(url)
print(response.status_code)

200


In [76]:
soup = BeautifulSoup(response.text, 'html.parser')

In [77]:
table = soup.find('table', id = 'example2')
print(table)

<table cellspacing="0" class="table table-striped table-bordered" id="example2" width="100%"> <thead> <tr> <th>#</th> <th>Country (or dependency)</th> <th>Population<br/> (2023)</th> <th>Yearly<br/> Change</th> <th>Net<br/> Change</th> <th>Density<br/> (P/Km²)</th> <th>Land Area<br/> (Km²)</th> <th>Migrants<br/> (net)</th> <th>Fert.<br/> Rate</th> <th>Med.<br/> Age</th> <th>Urban<br/> Pop %</th> <th>World<br/> Share</th> </tr> </thead> <tbody> <tr> <td>1</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/india-population/">India</a></td> <td style="font-weight: bold;">1,428,627,663</td> <td>0.81 %</td> <td>11,454,490</td> <td>481</td> <td>2,973,190</td> <td>-486,136</td> <td>2.0</td> <td>28</td> <td>36 %</td> <td>17.76 %</td> </tr> <tr> <td>2</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/china-population/">China</a></td> <td style="font-weight: bold;">1,425,671,352</td> <td>-0.02 %</td> <td>-21

In [78]:
for employee_data in table.find_all('thead'):#[0].find_all('tr'):
    row=employee_data.find_all('tr')[0]
print(row)

<tr> <th>#</th> <th>Country (or dependency)</th> <th>Population<br/> (2023)</th> <th>Yearly<br/> Change</th> <th>Net<br/> Change</th> <th>Density<br/> (P/Km²)</th> <th>Land Area<br/> (Km²)</th> <th>Migrants<br/> (net)</th> <th>Fert.<br/> Rate</th> <th>Med.<br/> Age</th> <th>Urban<br/> Pop %</th> <th>World<br/> Share</th> </tr>


In [79]:
header=[]
for i in range(12):
    header.append(row.find_all('th')[i].text)
header

['#',
 'Country (or dependency)',
 'Population (2023)',
 'Yearly Change',
 'Net Change',
 'Density (P/Km²)',
 'Land Area (Km²)',
 'Migrants (net)',
 'Fert. Rate',
 'Med. Age',
 'Urban Pop %',
 'World Share']

In [80]:
for employee_data in table.find_all('tbody'):
    rows = employee_data.find_all('tr')
print(rows)

[<tr> <td>1</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/india-population/">India</a></td> <td style="font-weight: bold;">1,428,627,663</td> <td>0.81 %</td> <td>11,454,490</td> <td>481</td> <td>2,973,190</td> <td>-486,136</td> <td>2.0</td> <td>28</td> <td>36 %</td> <td>17.76 %</td> </tr>, <tr> <td>2</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/china-population/">China</a></td> <td style="font-weight: bold;">1,425,671,352</td> <td>-0.02 %</td> <td>-215,985</td> <td>152</td> <td>9,388,211</td> <td>-310,220</td> <td>1.2</td> <td>39</td> <td>65 %</td> <td>17.72 %</td> </tr>, <tr> <td>3</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/us-population/">United States</a></td> <td style="font-weight: bold;">339,996,563</td> <td>0.50 %</td> <td>1,706,706</td> <td>37</td> <td>9,147,420</td> <td>999,700</td> <td>1.7</td> <td>38</td> <td>83 %</td> <td>4.2

In [81]:
data=[]
for row in rows:
    row_data=[]
    for i in range(12):
        row_data.append(row.find_all('td')[i].text)
    data.append(row_data)

df=pd.DataFrame(data,columns=header)
print(df.shape)
df.head(10)

(234, 12)


,#,Country (or dependency),Population (2023),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,1,India,"1,428,627,663",0.81 %,"11,454,490",481,"2,973,190","-486,136",2.0,28,36 %,17.76 %
1,2,China,"1,425,671,352",-0.02 %,"-215,985",152,"9,388,211","-310,220",1.2,39,65 %,17.72 %
2,3,United States,"339,996,563",0.50 %,"1,706,706",37,"9,147,420","999,700",1.7,38,83 %,4.23 %
3,4,Indonesia,"277,534,122",0.74 %,"2,032,783",153,"1,811,570","-49,997",2.1,30,59 %,3.45 %
4,5,Pakistan,"240,485,658",1.98 %,"4,660,796",312,"770,880","-165,988",3.3,21,35 %,2.99 %
5,6,Nigeria,"223,804,632",2.41 %,"5,263,420",246,"910,770","-59,996",5.1,17,54 %,2.78 %
6,7,Brazil,"216,422,446",0.52 %,"1,108,948",26,"8,358,140","6,000",1.6,34,88 %,2.69 %
7,8,Bangladesh,"172,954,319",1.03 %,"1,767,947","1,329","130,170","-309,977",1.9,27,41 %,2.15 %
8,9,Russia,"144,444,359",-0.19 %,"-268,955",9,"16,376,870","-136,414",1.5,39,75 %,1.80 %
9,10,Mexico,"128,455,567",0.75 %,"951,442",66,"1,943,950","-50,239",1.8,30,88 %,1.60 %


In [82]:
df.describe()

,#,Country (or dependency),Population (2023),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
count,234,234,234,234,234,234,234,234,234,234,234,234
unique,234,234,234,176,234,168,225,144,46,38,78,69
top,1,India,"1,428,627,663",0.74 %,"11,454,490",4,460,0,1.6,40,N.A.,0.00 %
freq,1,1,1,5,1,6,3,26,25,12,18,56


In [83]:
df.columns=df.columns.str.replace("\n",'').str.replace(" ","")
df.rename({'UrbanPop%':'UrbanPop(%)',
           'YearlyChange':'YearlyChange(%)',
           'WorldShare':'WorldShare(%)',
          'Fert.Rate':'FertRate',
          'Med.Age':'MedAge'}, axis=1, inplace=True)
df.columns

Index(['#', 'Country(ordependency)', 'Population(2023)', 'YearlyChange(%)',
       'NetChange', 'Density(P/Km²)', 'LandArea(Km²)', 'Migrants(net)',
       'FertRate', 'MedAge', 'UrbanPop(%)', 'WorldShare(%)'],
      dtype='object')

In [84]:
df.dtypes

#                        object
Country(ordependency)    object
Population(2023)         object
YearlyChange(%)          object
NetChange                object
Density(P/Km²)           object
LandArea(Km²)            object
Migrants(net)            object
FertRate                 object
MedAge                   object
UrbanPop(%)              object
WorldShare(%)            object
dtype: object

In [85]:
cols=['Population(2023)','LandArea(Km²)','NetChange','Migrants(net)','Density(P/Km²)']
for col in cols:
    df[col]=df[col].apply(lambda x: x.replace(",",""))


cols=['YearlyChange(%)','UrbanPop(%)','WorldShare(%)']
for col in cols:
    df[col]=df[col].apply(lambda x: x.replace("%","").strip())
df.head()

,#,Country(ordependency),Population(2023),YearlyChange(%),NetChange,Density(P/Km²),LandArea(Km²),Migrants(net),FertRate,MedAge,UrbanPop(%),WorldShare(%)
0,1,India,1428627663,0.81,11454490,481,2973190,-486136,2.0,28,36,17.76
1,2,China,1425671352,-0.02,-215985,152,9388211,-310220,1.2,39,65,17.72
2,3,United States,339996563,0.50,1706706,37,9147420,999700,1.7,38,83,4.23
3,4,Indonesia,277534122,0.74,2032783,153,1811570,-49997,2.1,30,59,3.45
4,5,Pakistan,240485658,1.98,4660796,312,770880,-165988,3.3,21,35,2.99


In [86]:
df.isna().sum()

#                        0
Country(ordependency)    0
Population(2023)         0
YearlyChange(%)          0
NetChange                0
Density(P/Km²)           0
LandArea(Km²)            0
Migrants(net)            0
FertRate                 0
MedAge                   0
UrbanPop(%)              0
WorldShare(%)            0
dtype: int64

In [87]:
df.loc[df['UrbanPop(%)']=='N.A.','UrbanPop(%)']=None
df.isna().sum()

#                         0
Country(ordependency)     0
Population(2023)          0
YearlyChange(%)           0
NetChange                 0
Density(P/Km²)            0
LandArea(Km²)             0
Migrants(net)             0
FertRate                  0
MedAge                    0
UrbanPop(%)              18
WorldShare(%)             0
dtype: int64

In [93]:
df.loc[df['FertRate']=='','FertRate']=None
df.loc[df['MedAge']=='','MedAge']=None
df.isna().sum()

#                         0
Country(ordependency)     0
Population(2023)          0
YearlyChange(%)           0
NetChange                 0
Density(P/Km²)            0
LandArea(Km²)             0
Migrants(net)             0
FertRate                  1
MedAge                    1
UrbanPop(%)              18
WorldShare(%)             0
dtype: int64

In [95]:
cols=['LandArea(Km²)','YearlyChange(%)','UrbanPop(%)','WorldShare(%)','FertRate','MedAge']
for col in cols:
    df[col]=df[col].astype('float')

cols=['Migrants(net)','Density(P/Km²)','NetChange','Population(2023)',]
for col in cols:
    df[col]=df[col].astype('int',)
df.dtypes

#                         object
Country(ordependency)     object
Population(2023)           int64
YearlyChange(%)          float64
NetChange                  int64
Density(P/Km²)             int64
LandArea(Km²)            float64
Migrants(net)              int64
FertRate                 float64
MedAge                   float64
UrbanPop(%)              float64
WorldShare(%)            float64
dtype: object

In [115]:
df['density_computed']=round(df['Population(2023)']/df['LandArea(Km²)'])
df.loc[df['density_computed']==float('inf'),'density_computed']=df.loc[df['density_computed']==float('inf'),'Density(P/Km²)']
df['density_computed']=df['density_computed'].astype('int')

df.head(10)

,#,Country(ordependency),Population(2023),YearlyChange(%),NetChange,Density(P/Km²),LandArea(Km²),Migrants(net),FertRate,MedAge,UrbanPop(%),WorldShare(%),density_computed
0,1,India,1428627663,0.81,11454490,481,2973190.0,-486136,2.0,28.0,36.0,17.76,481
1,2,China,1425671352,-0.02,-215985,152,9388211.0,-310220,1.2,39.0,65.0,17.72,152
2,3,United States,339996563,0.50,1706706,37,9147420.0,999700,1.7,38.0,83.0,4.23,37
3,4,Indonesia,277534122,0.74,2032783,153,1811570.0,-49997,2.1,30.0,59.0,3.45,153
4,5,Pakistan,240485658,1.98,4660796,312,770880.0,-165988,3.3,21.0,35.0,2.99,312
5,6,Nigeria,223804632,2.41,5263420,246,910770.0,-59996,5.1,17.0,54.0,2.78,246
6,7,Brazil,216422446,0.52,1108948,26,8358140.0,6000,1.6,34.0,88.0,2.69,26
7,8,Bangladesh,172954319,1.03,1767947,1329,130170.0,-309977,1.9,27.0,41.0,2.15,1329
8,9,Russia,144444359,-0.19,-268955,9,16376870.0,-136414,1.5,39.0,75.0,1.80,9
9,10,Mexico,128455567,0.75,951442,66,1943950.0,-50239,1.8,30.0,88.0,1.60,66


In [116]:
df[df['density_computed']==float('inf')]

,#,Country(ordependency),Population(2023),YearlyChange(%),NetChange,Density(P/Km²),LandArea(Km²),Migrants(net),FertRate,MedAge,UrbanPop(%),WorldShare(%),density_computed


In [117]:
df.describe()

,Population(2023),YearlyChange(%),NetChange,Density(P/Km²),LandArea(Km²),Migrants(net),FertRate,MedAge,UrbanPop(%),WorldShare(%),density_computed
count,2.340000e+02,234.000000,2.340000e+02,234.000000,2.340000e+02,2.340000e+02,233.000000,233.000000,216.000000,234.000000,234.00000
mean,3.437565e+07,0.965470,3.000230e+05,477.414530,5.559568e+05,1.301282e+01,2.414163,31.309013,59.550926,0.427393,528.42735
std,1.373861e+08,1.242413,1.001815e+06,2320.693692,1.691024e+06,1.698334e+05,1.155913,9.628386,23.826450,1.707829,2905.45229
min,5.180000e+02,-7.450000,-2.957105e+06,0.000000,0.000000e+00,-9.104750e+05,0.800000,15.000000,0.000000,0.000000,0.00000
25%,4.696482e+05,0.222500,2.360000e+02,38.250000,2.650000e+03,-9.776750e+03,1.600000,22.000000,40.750000,0.010000,38.25000
50%,5.643895e+06,0.805000,2.860150e+04,96.500000,7.972000e+04,-5.000000e+02,2.000000,32.000000,61.500000,0.070000,96.50000
75%,2.324537e+07,1.685000,2.236855e+05,242.000000,4.070800e+05,4.750000e+02,3.000000,40.000000,79.250000,0.290000,242.00000
max,1.428628e+09,4.980000,1.145449e+07,24360.000000,1.637687e+07,1.784718e+06,6.700000,54.000000,99.000000,17.760000,36297.00000


In [118]:
df[df['Density(P/Km²)']!=df['density_computed']]

,#,Country(ordependency),Population(2023),YearlyChange(%),NetChange,Density(P/Km²),LandArea(Km²),Migrants(net),FertRate,MedAge,UrbanPop(%),WorldShare(%),density_computed
214,215,Monaco,36297,-0.47,-172,24360,1.0,200,2.1,54.0,NaN,0.0,36297


In [122]:
#Save
DATAPATH='../assets'
FILENAME='country_population_data.csv'
df.to_csv(os.path.join(DATAPATH,FILENAME),sep=',')

## Scrap Country Codes data

In [15]:
url = 'https://docs.dnb.com/partner/en-US/iso_country_codes'
response = requests.get(url)
print(response.status_code)

200


In [16]:
soup = BeautifulSoup(response.text, 'html.parser')

In [17]:
table = soup.find('table', class_= 'TableStyle-dnb_tablestyle_2')
print(table)

<table class="TableStyle-dnb_tablestyle_2"><tbody><tr><td>WorldBase Country Name</td><td>WorldBase
    3 digit</td><td>WorldBase
    2 char</td><td>ISO
 Country Name</td><td>ISO
    3 digit</td><td>ISO
    3 char</td><td><span>ISO
    2 char</span></td><td>Comments</td></tr><tr><td>Aruba</td><td>034</td><td>AA</td><td>Aruba</td><td>533</td><td>ABW</td><td>AW</td><td> </td></tr><tr><td>Afghanistan</td><td>005</td><td>AF</td><td>Afghanistan</td><td>004</td><td>AFG</td><td>AF</td><td> </td></tr><tr><td>Angola</td><td>025</td><td>AO</td><td>Angola</td><td>024</td><td>AGO</td><td>AO</td><td> </td></tr><tr><td>Anguilla</td><td>027</td><td>AL</td><td>Anguilla</td><td>660</td><td>AIA</td><td>AI</td><td> </td></tr><tr><td>Albania</td><td>009</td><td>AB</td><td>Albania</td><td>008</td><td>ALB</td><td>AL</td><td> </td></tr><tr><td>Andorra</td><td>021</td><td>AN</td><td>Andorra</td><td>020</td><td>AND</td><td>AD</td><td> </td></tr><tr><td>Netherlands
 Antilles</td><td>525</td><td>NA</td><td>Nether

In [18]:
for country_codes in table.find_all('tbody'):
    rows = country_codes.find_all('tr')
print(rows)

[<tr><td>WorldBase Country Name</td><td>WorldBase
    3 digit</td><td>WorldBase
    2 char</td><td>ISO
 Country Name</td><td>ISO
    3 digit</td><td>ISO
    3 char</td><td><span>ISO
    2 char</span></td><td>Comments</td></tr>, <tr><td>Aruba</td><td>034</td><td>AA</td><td>Aruba</td><td>533</td><td>ABW</td><td>AW</td><td> </td></tr>, <tr><td>Afghanistan</td><td>005</td><td>AF</td><td>Afghanistan</td><td>004</td><td>AFG</td><td>AF</td><td> </td></tr>, <tr><td>Angola</td><td>025</td><td>AO</td><td>Angola</td><td>024</td><td>AGO</td><td>AO</td><td> </td></tr>, <tr><td>Anguilla</td><td>027</td><td>AL</td><td>Anguilla</td><td>660</td><td>AIA</td><td>AI</td><td> </td></tr>, <tr><td>Albania</td><td>009</td><td>AB</td><td>Albania</td><td>008</td><td>ALB</td><td>AL</td><td> </td></tr>, <tr><td>Andorra</td><td>021</td><td>AN</td><td>Andorra</td><td>020</td><td>AND</td><td>AD</td><td> </td></tr>, <tr><td>Netherlands
 Antilles</td><td>525</td><td>NA</td><td>Netherlands Antilles</td><td>530</td><td>

In [19]:
len(rows)

260

In [20]:
data=[]
header=[]
num_cols=8

#scrap header
for i in range(num_cols):
    header.append(rows[0].find_all('td')[i].text)

#scrap data
for r in range(1,len(rows)):
    row=rows[r]
    row_data=[]
    for i in range(num_cols):
        row_data.append(row.find_all('td')[i].text)
    data.append(row_data)

df_codes=pd.DataFrame(data,columns=header)
print(df_codes.shape)
df_codes.head(10)

(259, 8)


,WorldBase Country Name,WorldBase\n 3 digit,WorldBase\n 2 char,ISO\n Country Name,ISO\n 3 digit,ISO\n 3 char,ISO\n 2 char,Comments
0,Aruba,034,AA,Aruba,533,ABW,AW,
1,Afghanistan,005,AF,Afghanistan,004,AFG,AF,
2,Angola,025,AO,Angola,024,AGO,AO,
3,Anguilla,027,AL,Anguilla,660,AIA,AI,
4,Albania,009,AB,Albania,008,ALB,AL,
5,Andorra,021,AN,Andorra,020,AND,AD,
6,Netherlands\n Antilles,525,NA,Netherlands Antilles,530,ANT,AN,
7,United Arab\n Emirates,777,UA,United Arab Emirates,784,ARE,AE,D&B reports as separate countries
8,Ajman,898,XA,United Arab Emirates,784,ARE,AE,ISO includes as * UAE
9,Abu Dhabi,897,XB,United Arab Emirates,784,ARE,AE,ISO includes as * UAE


In [21]:
df_codes.columns=df_codes.columns.str.replace("\n",'').str.replace(" ","")
df_codes.columns

Index(['WorldBaseCountryName', 'WorldBase3digit', 'WorldBase2char',
       'ISOCountryName', 'ISO3digit', 'ISO3char', 'ISO2char', 'Comments'],
      dtype='object')

In [22]:
df_codes.head(10)

,WorldBaseCountryName,WorldBase3digit,WorldBase2char,ISOCountryName,ISO3digit,ISO3char,ISO2char,Comments
0,Aruba,034,AA,Aruba,533,ABW,AW,
1,Afghanistan,005,AF,Afghanistan,004,AFG,AF,
2,Angola,025,AO,Angola,024,AGO,AO,
3,Anguilla,027,AL,Anguilla,660,AIA,AI,
4,Albania,009,AB,Albania,008,ALB,AL,
5,Andorra,021,AN,Andorra,020,AND,AD,
6,Netherlands\n Antilles,525,NA,Netherlands Antilles,530,ANT,AN,
7,United Arab\n Emirates,777,UA,United Arab Emirates,784,ARE,AE,D&B reports as separate countries
8,Ajman,898,XA,United Arab Emirates,784,ARE,AE,ISO includes as * UAE
9,Abu Dhabi,897,XB,United Arab Emirates,784,ARE,AE,ISO includes as * UAE


## Read Data

In [132]:
DATAPATH='../assets'
ALIGNED='aligned'
EDA='eda'
CURATED='curated'
FILENAME='WEOOct2023all.xlsx'

In [133]:
df_imf=pd.read_excel(os.path.join(DATAPATH,FILENAME),engine='openpyxl')
#df_spain.rename({'Unnamed: 2':'Date'},axis=1,inplace=True)
#df_spain.set_index('Date', inplace=True)
print(df_imf.shape)
df_imf.head()

(8626, 59)


,WEO Country Code,ISO,WEO Subject Code,Country,Subject Descriptor,Subject Notes,Units,Scale,Country/Series-specific Notes,1980,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,Estimates Start After
0,512,AFG,NGDP_R,Afghanistan,"Gross domestic product, constant prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,NaN,...,1288.87,1021.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
1,512,AFG,NGDP_RPCH,Afghanistan,"Gross domestic product, constant prices",Annual percentages of constant price GDP are y...,Percent change,NaN,"See notes for: Gross domestic product, consta...",NaN,...,-2.351,-20.737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
2,512,AFG,NGDP,Afghanistan,"Gross domestic product, current prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,NaN,...,1547.29,1232.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
3,512,AFG,NGDPD,Afghanistan,"Gross domestic product, current prices",Values are based upon GDP in national currency...,U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",NaN,...,20.136,14.941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
4,512,AFG,PPPGDP,Afghanistan,"Gross domestic product, current prices",These data form the basis for the country weig...,Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",NaN,...,81.007,67.093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0


In [134]:
FILENAME='WDIEXCEL.xlsx'
df_country_codes=pd.read_excel(os.path.join(DATAPATH,FILENAME),engine='openpyxl',sheet_name='Country')
print(df_country_codes.shape)
df_country_codes

(265, 31)


,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,NaN,Latin America & Caribbean,High income,AW,...,NaN,Enhanced General Data Dissemination System (e-...,2020 (expected),NaN,NaN,Yes,NaN,NaN,2018.0,NaN
1,AFE,Africa Eastern and Southern,Africa Eastern and Southern,Africa Eastern and Southern,ZH,NaN,"26 countries, stretching from the Red Sea in t...",NaN,NaN,ZH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,The reporting period for national accounts dat...,South Asia,Low income,AF,...,Consolidated central government,Enhanced General Data Dissemination System (e-...,1979,"Demographic and Health Survey, 2015","Integrated household survey (IHS), 2016/17",NaN,NaN,NaN,2018.0,NaN
3,AFW,Africa Western and Central,Africa Western and Central,Africa Western and Central,ZI,NaN,"22 countries, stretching from the westernmost ...",NaN,NaN,ZI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,The World Bank systematically assesses the app...,Sub-Saharan Africa,Lower middle income,AO,...,Budgetary central government,Enhanced General Data Dissemination System (e-...,2014,"Demographic and Health Survey, 2015/16","Integrated household survey (IHS), 2008/09",NaN,NaN,NaN,2018.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,XKX,Kosovo,Kosovo,Republic of Kosovo,XK,Euro,NaN,Europe & Central Asia,Upper middle income,XK,...,NaN,Enhanced General Data Dissemination System (e-...,2011,"Multiple Indicator Cluster Survey, 2019/20","Expenditure survey/budget survey (ES/BS), 2015",NaN,2014,NaN,NaN,NaN
261,YEM,Yemen,"Yemen, Rep.",Republic of Yemen,YE,Yemeni rial,The World Bank systematically assesses the app...,Middle East & North Africa,Low income,RY,...,NaN,Enhanced General Data Dissemination System (e-...,2004,"Demographic and Health Survey, 2013","Expenditure survey/budget survey (ES/BS), 2014",NaN,NaN,2012.0,2015.0,NaN
262,ZAF,South Africa,South Africa,Republic of South Africa,ZA,South African rand,Fiscal year end: March 31; reporting period fo...,Sub-Saharan Africa,Upper middle income,ZA,...,Consolidated central government,Special Data Dissemination Standard (SDDS),2011,"Demographic and Health Survey, 2016","Expenditure survey/budget survey (ES/BS), 2014/15",NaN,2007,2010.0,2018.0,NaN
263,ZMB,Zambia,Zambia,Republic of Zambia,ZM,New Zambian kwacha,National accounts data were rebased to reflect...,Sub-Saharan Africa,Lower middle income,ZM,...,Budgetary central government,Enhanced General Data Dissemination System (e-...,2020 (expected),"Demographic and Health Survey, 2018","Integrated household survey (IHS), 2015",NaN,NaN,1994.0,2018.0,NaN


In [135]:
FILENAME='WDIEXCEL.xlsx'
df_series=pd.read_excel(os.path.join(DATAPATH,FILENAME),engine='openpyxl',sheet_name='Series')
print(df_series.shape)
df_series

(1492, 20)


,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,Limitations and exceptions,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type
0,AG.AGR.TRAC.NO,Environment: Agricultural production,"Agricultural machinery, tractors",NaN,Agricultural machinery refers to the number of...,NaN,Annual,NaN,NaN,Sum,The data are collected by the Food and Agricul...,NaN,NaN,"Food and Agriculture Organization, electronic ...",A tractor provides the power and traction to m...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0
1,AG.CON.FERT.PT.ZS,Environment: Agricultural production,Fertilizer consumption (% of fertilizer produc...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,The world and regional aggregate series do not...,Weighted average,The FAO has revised the time series for fertil...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0
2,AG.CON.FERT.ZS,Environment: Agricultural production,Fertilizer consumption (kilograms per hectare ...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,The world and regional aggregate series do not...,Weighted average,The FAO has revised the time series for fertil...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0
3,AG.LND.AGRI.K2,Environment: Land use,Agricultural land (sq. km),NaN,Agricultural land refers to the share of land ...,NaN,Annual,NaN,Areas of former states are included in the suc...,Sum,The data are collected by the Food and Agricul...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Agricultural land constitutes only a part of a...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0
4,AG.LND.AGRI.ZS,Environment: Land use,Agricultural land (% of land area),NaN,Agricultural land refers to the share of land ...,NaN,Annual,NaN,Areas of former states are included in the suc...,Weighted average,The data are collected by the Food and Agricul...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Agriculture is still a major sector in many ec...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,VC.IDP.NWDS,Public Sector: Conflict & fragility,"Internally displaced persons, new displacement...",NaN,Internally displaced persons are defined accor...,NaN,Annual,NaN,NaN,Sum,Please note that most of the figures are estim...,NaN,NaN,The Internal Displacement Monitoring Centre (h...,"Internally displaced persons are ""persons or g...",Although all persons affected by conflict and/...,NaN,NaN,NaN,CC BY-4.0
1488,VC.IDP.TOCV,Public Sector: Conflict & fragility,"Internally displaced persons, total displaced ...",NaN,Internally displaced persons are defined accor...,NaN,Annual,NaN,NaN,Sum,Please note that most of the figures are estim...,NaN,NaN,The Internal Displacement Monitoring Centre (h...,"Internally displaced persons are ""persons or g...",Although all persons affected by conflict and/...,NaN,NaN,NaN,CC BY-4.0
1489,VC.IHR.PSRC.FE.P5,Public Sector: Conflict & fragility,"Intentional homicides, female (per 100,000 fem...",NaN,"Intentional homicides, female are estimates of...",NaN,Annual,NaN,NaN,NaN,Statistics reported to the United Nations in t...,NaN,NaN,UN Office on Drugs and Crime's International H...,The definitions used to produce data are in li...,"In some regions, organized crime, drug traffic...",NaN,NaN,NaN,CC BY-4.0
1490,VC.IHR.PSRC.MA.P5,Public Sector: Conflict & fragility,"Intentional homicides, male (per 100,000 male)",N

In [136]:
FILENAME='economic-regions.xlsx'
df_regions=pd.read_excel(os.path.join(DATAPATH,ALIGNED,FILENAME),engine='openpyxl')
print(df_regions.shape)
df_regions

(12, 5)


,Unnamed: 0,Region_name,Countries,num_of_countries,countries_not_in_IMFdata
0,0,Advanced Economies,"Andorra,Australia,Austria,Belgium,Canada,Croat...",41,[]
1,1,Euro Area,"Austria,Belgium,Croatia,Cyprus,Estonia,Finland...",20,[]
2,2,Major Advanced Economies (G7),"Canada,France,Germany,Italy,Japan,United Kingd...",7,[]
3,3,Other Advanced Economies \n(Advanced Economies...,"Andorra,Australia,Czech Republic,Denmark,Hong ...",17,[]
4,4,European Union,"Austria,Belgium,Bulgaria,Croatia,Cyprus,Czech ...",27,[]
5,5,ASEAN-5,"Indonesia,Malaysia,Philippines,Singapore,Thailand",5,[]
6,6,Emerging market and developing economies,"Afghanistan,Albania,Algeria,Angola,Antigua and...",155,[]
7,7,Emerging and developing Asia,"Bangladesh,Bhutan,Brunei Darussalam,Cambodia,C...",30,[]
8,8,Emerging and developing Europe,"Albania,Belarus,Bosnia and Herzegovina,Bulgari...",15,[]
9,9,Latin America and the Caribbean,"Antigua and Barbuda,Argentina,Aruba,The Bahama...",33,[]


In [129]:
df_regions['Region_name'].unique()

array(['Advanced Economies', 'Euro Area', 'Major Advanced Economies (G7)',
       'Other Advanced Economies \n(Advanced Economies excluding G7 and Euro Area)',
       'European Union', 'ASEAN-5',
       'Emerging market and developing economies',
       'Emerging and developing Asia', 'Emerging and developing Europe',
       'Latin America and the Caribbean', 'Middle East and Central Asia',
       'Sub-Saharan Africa'], dtype=object)

In [ ]:
['Latin America & Caribbean', None, 'South Asia',
       'Sub-Saharan Africa', 'Europe & Central Asia',
       'Middle East & North Africa', 'East Asia & Pacific',
       'North America']

In [28]:
df_imf['Country'].nunique()

196

In [29]:
df_imf['Country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'The Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Democratic Republic of the Congo', 'Republic of Congo',
       'Costa Rica', "CÙte d'Ivoire", 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'The Gambia', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Grenada', 'Guate

In [137]:
df_imf.drop([8624,8625],axis=0,inplace=True)
df_imf['WEO Country Code'].unique()

array([512, 914, 612, 171, 614, 311, 213, 911, 314, 193, 122, 912, 313,
       419, 513, 316, 913, 124, 339, 638, 514, 218, 963, 616, 223, 516,
       918, 748, 618, 624, 522, 622, 156, 626, 628, 228, 924, 233, 632,
       636, 634, 238, 662, 960, 423, 935, 128, 611, 321, 243, 248, 469,
       253, 642, 643, 939, 734, 644, 819, 172, 132, 646, 648, 915, 134,
       652, 174, 328, 258, 656, 654, 336, 263, 268, 532, 944, 176, 534,
       536, 429, 433, 178, 436, 136, 343, 158, 439, 916, 664, 826, 542,
       967, 443, 917, 544, 941, 446, 666, 668, 672, 946, 137, 546, 674,
       676, 548, 556, 678, 181, 867, 682, 684, 273, 868, 921, 948, 943,
       686, 688, 518, 728, 836, 558, 138, 196, 278, 692, 694, 962, 142,
       449, 564, 565, 283, 853, 288, 293, 566, 964, 182, 359, 453, 968,
       922, 714, 862, 135, 716, 456, 722, 942, 718, 724, 576, 936, 961,
       813, 726, 199, 733, 184, 524, 361, 362, 364, 732, 366, 144, 146,
       463, 528, 923, 738, 578, 537, 742, 866, 369, 744, 186, 92

In [138]:
df_imf[df_imf['WEO Country Code'].isna()]

,WEO Country Code,ISO,WEO Subject Code,Country,Subject Descriptor,Subject Notes,Units,Scale,Country/Series-specific Notes,1980,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,Estimates Start After


In [32]:
set(df['Country(ordependency)'].tolist())-set(df_imf['Country'].tolist())

{'American Samoa',
 'Anguilla',
 'Bahamas',
 'Bermuda',
 'British Virgin Islands',
 'Brunei ',
 'Caribbean Netherlands',
 'Cayman Islands',
 'Congo',
 'Cook Islands',
 'Cuba',
 'Curaçao',
 'Czech Republic (Czechia)',
 "Côte d'Ivoire",
 'DR Congo',
 'Faeroe Islands',
 'Falkland Islands',
 'French Guiana',
 'French Polynesia',
 'Gambia',
 'Gibraltar',
 'Greenland',
 'Guadeloupe',
 'Guam',
 'Holy See',
 'Hong Kong',
 'Iran',
 'Isle of Man',
 'Kyrgyzstan',
 'Laos',
 'Liechtenstein',
 'Macao',
 'Martinique',
 'Mayotte',
 'Monaco',
 'Montserrat',
 'New Caledonia',
 'Niue',
 'North Korea',
 'Northern Mariana Islands',
 'Réunion',
 'Saint Barthelemy',
 'Saint Helena',
 'Saint Kitts & Nevis',
 'Saint Lucia',
 'Saint Martin',
 'Saint Pierre & Miquelon',
 'Sao Tome & Principe',
 'Sint Maarten',
 'Slovakia',
 'South Korea',
 'St. Vincent & Grenadines',
 'State of Palestine',
 'Taiwan',
 'Tokelau',
 'Turkey',
 'Turks and Caicos',
 'U.S. Virgin Islands',
 'Wallis & Futuna',
 'Western Sahara'}

In [33]:
set(df_imf['Country'].tolist())-set(df['Country(ordependency)'].tolist())

{'Brunei Darussalam',
 'Czech Republic',
 "CÙte d'Ivoire",
 'Democratic Republic of the Congo',
 'Hong Kong SAR',
 'Islamic Republic of Iran',
 'Korea',
 'Kosovo',
 'Kyrgyz Republic',
 'Lao P.D.R.',
 'Macao SAR',
 'Republic of Congo',
 'Slovak Republic',
 'St. Kitts and Nevis',
 'St. Lucia',
 'St. Vincent and the Grenadines',
 'S„o TomÈ and PrÌncipe',
 'Taiwan Province of China',
 'The Bahamas',
 'The Gambia',
 'T¸rkiye',
 'West Bank and Gaza'}

In [130]:
import wbgapi

data=[]
for elem in wbgapi.economy.list(labels=True,skipAggs=True):
    data.append((elem['id'],elem['value'], elem['longitude'],elem['latitude']))
df_wb=pd.DataFrame(data,columns=['id','countryName','longitude','latitude'])
print(df_wb.shape)
df_wb.tail()

(217, 4)


,id,countryName,longitude,latitude
212,XKX,Kosovo,20.9260,42.5650
213,YEM,"Yemen, Rep.",44.2075,15.3520
214,ZAF,South Africa,28.1871,-25.7460
215,ZMB,Zambia,28.2937,-15.3982
216,ZWE,Zimbabwe,31.0672,-17.8312


In [153]:
k=set(df_wb['id'].tolist())-set(df_imf['ISO'].tolist())
df_wb[df_wb['id'].isin(k)]

,id,countryName,longitude,latitude
8,ASM,American Samoa,-170.691000,-14.284600
24,BMU,Bermuda,-64.706000,32.329300
34,CHI,Channel Islands,NaN,NaN
45,CUB,Cuba,-82.366700,23.133300
46,CUW,Curacao,NaN,NaN
47,CYM,Cayman Islands,-81.385700,19.302200
65,FRO,Faroe Islands,-6.911810,61.892600
71,GIB,Gibraltar,NaN,NaN
78,GRL,Greenland,-51.721400,64.183600
80,GUM,Guam,144.794000,13.444300


In [152]:
len(k)

22

In [140]:
s=set(df_imf['ISO'].tolist())-set(df_wb['id'].tolist())
s

{'TWN', 'UVK', 'WBG'}

In [141]:
set(df_imf['Country'].tolist())-set(df_wb['countryName'].tolist())

{'Czech Republic',
 "CÙte d'Ivoire",
 'Democratic Republic of the Congo',
 'Egypt',
 'Hong Kong SAR',
 'Islamic Republic of Iran',
 'Korea',
 'Lao P.D.R.',
 'Macao SAR',
 'Micronesia',
 'Republic of Congo',
 'Russia',
 'Syria',
 'S„o TomÈ and PrÌncipe',
 'Taiwan Province of China',
 'The Bahamas',
 'The Gambia',
 'T¸rkiye',
 'Venezuela',
 'Vietnam',
 'Yemen'}

In [150]:
l=set(df_imf['Country'])
df_wb[~df_wb['countryName'].isin(l)]

,id,countryName,longitude,latitude
8,ASM,American Samoa,-170.691000,-14.284600
20,BHS,"Bahamas, The",-77.339000,25.066100
24,BMU,Bermuda,-64.706000,32.329300
34,CHI,Channel Islands,NaN,NaN
37,CIV,Cote d'Ivoire,-4.030500,5.332000
39,COD,"Congo, Dem. Rep.",15.322200,-4.325000
40,COG,"Congo, Rep.",15.266200,-4.276700
45,CUB,Cuba,-82.366700,23.133300
46,CUW,Curacao,NaN,NaN
47,CYM,Cayman Islands,-81.385700,19.302200


In [143]:
df_wb[df_wb['id'].isin(k)][40:]

,id,countryName,longitude,latitude


In [144]:
id_rename={'XKX':'UVK',
           'PSE':'WBG'
          }
df_wb['id'].replace(id_rename,inplace=True)

In [145]:
df_wb[df_wb['countryName']=='Kosovo']

,id,countryName,longitude,latitude
212,UVK,Kosovo,20.926,42.565


In [146]:
FILENAME='country_LatLong_data.xlsx'
df_wb.to_excel(os.path.join(DATAPATH,FILENAME))

In [57]:
countries_set=set(df_imf['Country'])
df_regions['num_of_countries']=df_regions['Countries'].apply(lambda x: len(x.strip().split(',')))

def find_countries_not_in_set(country_list,country_set):
    not_found=[]
    for country in country_list:
        country=country.strip()
        if country not in country_set:
            not_found.append(country)
    return not_found
df_regions['countries_not_in_IMFdata']=df_regions['Countries'].apply(lambda x: find_countries_not_in_set(x.split(','),countries_set))
df_regions

,Region_name,Countries,num_of_countries,countries_not_in_IMFdata
0,Advanced Economies,"Andorra, Australia, Austria, Belgium, Canada, ...",41,[]
1,Euro Area,"Austria, Belgium, Croatia, Cyprus, Estonia, Fi...",20,[]
2,Major Advanced Economies (G7),"Canada, France, Germany, Italy, Japan, United ...",7,[]
3,Other Advanced Economies \n(Advanced Economies...,"Andorra, Australia, Czech Republic, Denmark, H...",17,[]
4,European Union,"Austria, Belgium, Bulgaria, Croatia, Cyprus, C...",27,[]
5,ASEAN-5,"Indonesia, Malaysia, Philippines, Singapore, T...",5,[]
6,Emerging market and developing economies,"Afghanistan, Albania, Algeria, Angola, Antigua...",155,"[Côte d'Ivoire, São Tomé and Príncipe, Türkiye]"
7,Emerging and developing Asia,"Bangladesh, Bhutan, Brunei Darussalam, Cambodi...",30,[]
8,Emerging and developing Europe,"Albania, Belarus, Bosnia and Herzegovina, Bulg...",15,[Türkiye]
9,Latin America and the Caribbean,"Antigua and Barbuda, Argentina, Aruba, The Bah...",33,[]


In [154]:
df_wb[df_wb['id'].isin(k)]

,id,countryName,longitude,latitude
8,ASM,American Samoa,-170.691000,-14.284600
24,BMU,Bermuda,-64.706000,32.329300
34,CHI,Channel Islands,NaN,NaN
45,CUB,Cuba,-82.366700,23.133300
46,CUW,Curacao,NaN,NaN
47,CYM,Cayman Islands,-81.385700,19.302200
65,FRO,Faroe Islands,-6.911810,61.892600
71,GIB,Gibraltar,NaN,NaN
78,GRL,Greenland,-51.721400,64.183600
80,GUM,Guam,144.794000,13.444300


## Using Pycountry to Harmonize Country codes across datasets 

In [147]:
len(pycountry.countries)

249

In [175]:
pycountry.countries.search_fuzzy('GIB')

[Country(alpha_2='GI', alpha_3='GIB', flag='🇬🇮', name='Gibraltar', numeric='292'),
 Country(alpha_2='LR', alpha_3='LBR', flag='🇱🇷', name='Liberia', numeric='430', official_name='Republic of Liberia')]

In [109]:
%%time
df_data=wb.data.DataFrame(series_to_keep, economy='all',skipAggs=True,labels=True,numericTimeKeys=True,timeColumns=True)
df_data

CPU times: user 13.9 s, sys: 261 ms, total: 14.2 s
Wall time: 1min 44s


,,Country,Series,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
economy,series,,,,,,,,,,,,,,,,,,,,,
AZE,SP.POP.TOTL,Azerbaijan,"Population, total",3894500.0,4.045750e+06,4.168150e+06,4.293550e+06,4.439250e+06,4.574650e+06,4.708150e+06,4.832000e+06,...,9.535079e+06,9.649341e+06,9.757812e+06,9.854033e+06,9.939771e+06,1.002428e+07,1.009312e+07,1.013775e+07,1.014176e+07,1.011256e+07
CHL,SP.POP.TOTL,Chile,"Population, total",8141820.0,8.313535e+06,8.486120e+06,8.656006e+06,8.825046e+06,8.994936e+06,9.161649e+06,9.326268e+06,...,1.768711e+07,1.787012e+07,1.808388e+07,1.836858e+07,1.870145e+07,1.903948e+07,1.930032e+07,1.949318e+07,1.960373e+07,1.962959e+07
ZAF,SP.POP.TOTL,South Africa,"Population, total",16520441.0,1.698946e+07,1.750313e+07,1.804222e+07,1.860310e+07,1.918719e+07,1.978977e+07,2.041068e+07,...,5.472955e+07,5.587650e+07,5.642227e+07,5.664121e+07,5.733964e+07,5.808706e+07,5.880193e+07,5.939226e+07,5.989388e+07,6.041450e+07
BEN,SP.POP.TOTL,Benin,"Population, total",2512284.0,2.551216e+06,2.593302e+06,2.638082e+06,2.685619e+06,2.735843e+06,2.788642e+06,2.843739e+06,...,1.061484e+07,1.093278e+07,1.126008e+07,1.159678e+07,1.194068e+07,1.229044e+07,1.264312e+07,1.299690e+07,1.335286e+07,1.371283e+07
MUS,SP.POP.TOTL,Mauritius,"Population, total",676683.0,6.807570e+05,7.003490e+05,7.188610e+05,7.363810e+05,7.530000e+05,7.688130e+05,7.839170e+05,...,1.261208e+06,1.262879e+06,1.263747e+06,1.264887e+06,1.265577e+06,1.265985e+06,1.266014e+06,1.266334e+06,1.262523e+06,1.261041e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRK,NY.GDP.MKTP.KD.ZG,"Korea, Dem. People's Rep.",GDP growth (annual %),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LBY,NY.GDP.MKTP.KD.ZG,Libya,GDP growth (annual %),NaN,9.895804e+00,3.252464e+01,3.502038e+01,4.507169e+01,2.814888e+01,1.859216e+01,1.111009e+01,...,-2.304281e+01,-8.426563e-01,-1.490941e+00,3.249180e+01,7.941368e+00,-5.994784e+00,-5.831823e+01,1.534926e+02,1.336148e+00,-1.672590e+00
BFA,NY.GDP.MKTP.KD.ZG,Burkina Faso,GDP growth (annual %),NaN,4.043928e+00,6.128799e+00,-1.268016e+00,2.282819e+00,3.764545e+00,5.376131e-01,8.823315e+00,...,4.326846e+00,3.921229e+00,5.957977e+00,6.203489e+00,6.604569e+00,5.889205e+00,2.010773e+00,6.939155e+00,1.777915e+00,2.960180e+00


In [49]:
tmp_set={'STP'}
fuzzy_search_word='São Tomé and Príncipe'
new_name=pycountry.countries.search_fuzzy(fuzzy_search_word)[0].name
df_imf.loc[df_imf['ISO'].isin(tmp_set),'Country']=new_name
df_imf[df_imf['ISO'].isin(tmp_set)]

,WEO Country Code,ISO,WEO Subject Code,Country,Subject Descriptor,Subject Notes,Units,Scale,Country/Series-specific Notes,1980,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,Estimates Start After
6468,716,STP,NGDP_R,Sao Tome and Principe,"Gross domestic product, constant prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office. Instituto ...,1.828,...,4.074,4.151,4.154,4.175,4.275,4.407,4.566,4.733,4.907,2020.0
6469,716,STP,NGDP_RPCH,Sao Tome and Principe,"Gross domestic product, constant prices",Annual percentages of constant price GDP are y...,Percent change,NaN,"See notes for: Gross domestic product, consta...",-1.08,...,2.625,1.899,0.065,0.5,2.4,3.1,3.6,3.65,3.69,2020.0
6470,716,STP,NGDP,Sao Tome and Principe,"Gross domestic product, current prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office. Instituto ...,0.003,...,10.211,10.942,12.734,15.097,16.818,18.348,19.926,21.65,23.531,2020.0
6471,716,STP,NGDPD,Sao Tome and Principe,"Gross domestic product, current prices",Values are based upon GDP in national currency...,U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",0.069,...,0.476,0.529,0.548,0.674,0.76,0.832,0.906,0.985,1.07,2020.0
6472,716,STP,PPPGDP,Sao Tome and Principe,"Gross domestic product, current prices",These data form the basis for the country weig...,Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",0.144,...,0.81,0.862,0.923,0.962,1.007,1.06,1.119,1.181,1.247,2020.0
6473,716,STP,NGDP_D,Sao Tome and Principe,"Gross domestic product, deflator",The GDP deflator is derived by dividing curren...,Index,NaN,"See notes for: Gross domestic product, consta...",0.156,...,250.644,263.588,306.562,361.629,393.412,416.307,436.399,457.451,479.51,2020.0
6474,716,STP,NGDPRPC,Sao Tome and Principe,"Gross domestic product per capita, constant pr...",GDP is expressed in constant national currency...,National currency,Units,"See notes for: Gross domestic product, consta...",20646.51,...,18687.25,18615.22,18299.13,17994.28,18037.4,18212.34,17766.81,18051.32,18349.42,2011.0
6475,716,STP,NGDPRPPPPC,Sao Tome and Principe,"Gross domestic product per capita, constant pr...",GDP is expressed in constant international dol...,Purchasing power parity; 2017 international do...,Units,"See notes for: Gross domestic product, consta...",3886.49,...,3517.67,3504.12,3444.62,3387.23,3395.35,3428.28,3344.41,3397.97,3454.08,2011.0
6476,716,STP,NGDPPC,Sao Tome and Principe,"Gross domestic product per capita, current prices",GDP is expressed in current national currency ...,National currency,Units,"See notes for: Gross domestic product, curren...",32.301,...,46838.4,49067.44,56098.22,65072.47,70961.39,75819.31,77534.25,82575.94,87987.22,2011.0
6477,716,STP,NGDPDPC,Sao Tome and Principe,"Gross domestic product per capita, current prices",GDP is expressed in current U.S. dollars per p...,U.S. dollars,Units,"See notes for: Gross domestic product, curren...",775.709,...,2181.87,2370.31,2413.09,2904.72,3205.9,3439.79,3526.02,3755.3,4001.39,2011.0


In [65]:
tmp_set={'CIV'}
fuzzy_search_word='Côte d\'Ivoire'
new_name=pycountry.countries.search_fuzzy(fuzzy_search_word)[0].name
df_imf.loc[df_imf['ISO'].isin(tmp_set),'Country']=new_name
df_imf[df_imf['ISO'].isin(tmp_set)]

,WEO Country Code,ISO,WEO Subject Code,Country,Subject Descriptor,Subject Notes,Units,Scale,Country/Series-specific Notes,1980,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,Estimates Start After
1848,662,CIV,NGDP_R,Côte d'Ivoire,"Gross domestic product, constant prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,12828.21,...,35426.54,37906.4,40446.13,42953.79,45788.74,48719.22,51788.53,54947.63,58244.49,2020.0
1849,662,CIV,NGDP_RPCH,Côte d'Ivoire,"Gross domestic product, constant prices",Annual percentages of constant price GDP are y...,Percent change,NaN,"See notes for: Gross domestic product, consta...",5.196,...,1.738,7,6.7,6.2,6.6,6.4,6.3,6.1,6,2020.0
1850,662,CIV,NGDP,Côte d'Ivoire,"Gross domestic product, current prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,2932.03,...,36252,39821.44,43681.48,47874.2,52105.61,56466.02,61043.78,65768.1,71108.47,2020.0
1851,662,CIV,NGDPD,Côte d'Ivoire,"Gross domestic product, current prices",Values are based upon GDP in national currency...,U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",13.877,...,63.074,71.849,70.18,79.43,86.892,94.471,102.309,109.813,118.261,2020.0
1852,662,CIV,PPPGDP,Côte d'Ivoire,"Gross domestic product, current prices",These data form the basis for the country weig...,Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",20.664,...,144.177,161.199,184.048,202.647,220.916,239.792,259.846,280.737,303.095,2020.0
1853,662,CIV,NGDP_D,Côte d'Ivoire,"Gross domestic product, deflator",The GDP deflator is derived by dividing curren...,Index,NaN,"See notes for: Gross domestic product, consta...",22.856,...,102.33,105.052,107.999,111.455,113.796,115.901,117.871,119.692,122.086,2020.0
1854,662,CIV,NGDPRPC,Côte d'Ivoire,"Gross domestic product per capita, constant pr...",GDP is expressed in constant national currency...,National currency,Units,"See notes for: Gross domestic product, consta...",1604958.82,...,1314149.14,1370506.41,1425273.24,1475282.82,1532798.72,1589569.04,1646892.68,1703073.23,1759510.36,2015.0
1855,662,CIV,NGDPRPPPPC,Côte d'Ivoire,"Gross domestic product per capita, constant pr...",GDP is expressed in constant international dol...,Purchasing power parity; 2017 international do...,Units,"See notes for: Gross domestic product, consta...",6185.34,...,5064.59,5281.78,5492.85,5685.58,5907.24,6126.03,6346.94,6563.46,6780.96,2015.0
1856,662,CIV,NGDPPC,Côte d'Ivoire,"Gross domestic product per capita, current prices",GDP is expressed in current national currency ...,National currency,Units,"See notes for: Gross domestic product, curren...",366830.85,...,1344769.53,1439744.86,1539283.14,1644278.45,1744258.92,1842325.03,1941212.64,2038448.18,2148119.08,2015.0
1857,662,CIV,NGDPDPC,Côte d'Ivoire,"Gross domestic product per capita, current prices",GDP is expressed in current U.S. dollars per p...,U.S. dollars,Units,"See notes for: Gross domestic product, curren...",1736.23,...,2339.73,2597.7,2473.05,2728.08,2908.75,3082.33,3253.45,3403.6,3572.55,2015.0


In [66]:
tmp_set={'TUR'}
fuzzy_search_word='Türkiye'
new_name=pycountry.countries.search_fuzzy(fuzzy_search_word)[0].name
df_imf.loc[df_imf['ISO'].isin(tmp_set),'Country']=new_name
df_imf[df_imf['ISO'].isin(tmp_set)]

,WEO Country Code,ISO,WEO Subject Code,Country,Subject Descriptor,Subject Notes,Units,Scale,Country/Series-specific Notes,1980,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,Estimates Start After
7876,186,TUR,NGDP_R,Türkiye,"Gross domestic product, constant prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office. Data from ...,308.657,...,1804.39,2010.8,2122.07,2206.04,2271.87,2344.18,2419.69,2497.84,2578.64,2022.0
7877,186,TUR,NGDP_RPCH,Türkiye,"Gross domestic product, constant prices",Annual percentages of constant price GDP are y...,Percent change,NaN,"See notes for: Gross domestic product, consta...",-0.779,...,1.86,11.439,5.533,3.957,2.984,3.183,3.221,3.23,3.235,2022.0
7878,186,TUR,NGDP,Türkiye,"Gross domestic product, current prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office. Data from ...,0.007,...,5048.57,7256.14,15011.78,24563.93,40113.32,63085.56,94991.77,134909.85,188874.85,2022.0
7879,186,TUR,NGDPD,Türkiye,"Gross domestic product, current prices",Values are based upon GDP in national currency...,U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",96.596,...,720.159,818.337,905.841,1154.6,1340.69,1402.11,1454.23,1515.51,1576,2022.0
7880,186,TUR,PPPGDP,Türkiye,"Gross domestic product, current prices",These data form the basis for the country weig...,Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",159.188,...,2549.64,2968.93,3352.7,3613.54,3805.67,4005.96,4215.23,4430.94,4659.02,2022.0
7881,186,TUR,NGDP_D,Türkiye,"Gross domestic product, deflator",The GDP deflator is derived by dividing curren...,Index,NaN,"See notes for: Gross domestic product, consta...",0.002,...,279.794,360.858,707.413,1113.49,1765.65,2691.15,3925.78,5401.06,7324.6,2022.0
7882,186,TUR,NGDPRPC,Türkiye,"Gross domestic product per capita, constant pr...",GDP is expressed in constant national currency...,National currency,Units,"See notes for: Gross domestic product, consta...",6817.86,...,21579.99,23745.87,24883.52,25572.02,26042.33,26580.91,27149.97,27742.88,28359.64,2022.0
7883,186,TUR,NGDPRPPPPC,Türkiye,"Gross domestic product per capita, constant pr...",GDP is expressed in constant international dol...,Purchasing power parity; 2017 international do...,Units,"See notes for: Gross domestic product, consta...",9122.79,...,28875.58,31773.68,33295.94,34217.21,34846.51,35567.17,36328.6,37121.97,37947.24,2022.0
7884,186,TUR,NGDPPC,Türkiye,"Gross domestic product per capita, current prices",GDP is expressed in current national currency ...,National currency,Units,"See notes for: Gross domestic product, curren...",0.162,...,60379.46,85688.97,176029.27,284740.82,459817.42,715333.36,1065848.35,1498408.91,2077229.93,2022.0
7885,186,TUR,NGDPDPC,Türkiye,"Gross domestic product per capita, current prices",GDP is expressed in current U.S. dollars per p...,U.S. dollars,Units,"See notes for: Gross domestic product, curren...",2133.69,...,8612.9,9663.88,10621.97,13383.92,15368.27,15898.61,16317.09,16832.41,17332.69,2022.0


In [68]:
countries_set=set(df_imf['Country'])

def rename_country_names(country_list,country_set):
    new_list=[]
    for country in country_list:
        country=country.strip()
        if country in country_set:
            new_list.append(country)
        else:
            new_name=pycountry.countries.search_fuzzy(country)[0].name
            new_list.append(new_name)
    return ','.join(new_list)

df_regions['Countries']=df_regions['Countries'].apply(lambda x: rename_country_names(x.split(','),countries_set))
df_regions['countries_not_in_IMFdata']=df_regions['Countries'].apply(lambda x: find_countries_not_in_set(x.split(','),countries_set))
df_regions

,Region_name,Countries,num_of_countries,countries_not_in_IMFdata
0,Advanced Economies,"Andorra,Australia,Austria,Belgium,Canada,Croat...",41,[]
1,Euro Area,"Austria,Belgium,Croatia,Cyprus,Estonia,Finland...",20,[]
2,Major Advanced Economies (G7),"Canada,France,Germany,Italy,Japan,United Kingd...",7,[]
3,Other Advanced Economies \n(Advanced Economies...,"Andorra,Australia,Czech Republic,Denmark,Hong ...",17,[]
4,European Union,"Austria,Belgium,Bulgaria,Croatia,Cyprus,Czech ...",27,[]
5,ASEAN-5,"Indonesia,Malaysia,Philippines,Singapore,Thailand",5,[]
6,Emerging market and developing economies,"Afghanistan,Albania,Algeria,Angola,Antigua and...",155,[]
7,Emerging and developing Asia,"Bangladesh,Bhutan,Brunei Darussalam,Cambodia,C...",30,[]
8,Emerging and developing Europe,"Albania,Belarus,Bosnia and Herzegovina,Bulgari...",15,[]
9,Latin America and the Caribbean,"Antigua and Barbuda,Argentina,Aruba,The Bahama...",33,[]


The above df shows that countries within the regions have been harmonized with the IMF data. Even country names within the IMF data have been fixed.

## Save to file

In [69]:
FILENAME='WEO_Oct2023_country_region.xlsx'
df_imf.to_excel(os.path.join(DATAPATH,FILENAME))

In [70]:
FILENAME='economic-regions.xlsx'
df_regions.to_excel(os.path.join(DATAPATH,FILENAME))

## Working with Geopandas

In [117]:
# !pip install pyogrio fiona geopandas -y
# !conda install geopandas -y

In [1]:
import geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world

/var/folders/ld/f4rffqnd77b6qvbx5wq1rns40000gn/T/ipykernel_10663/2329397954.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))


,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...
172,6944975.0,Europe,Serbia,SRB,51475,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,622137.0,Europe,Montenegro,MNE,5542,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,1794248.0,Europe,Kosovo,-99,7926,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,1394973.0,North America,Trinidad and Tobago,TTO,24269,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


In [2]:
world['continent'].unique()

array(['Oceania', 'Africa', 'North America', 'Asia', 'South America',
       'Europe', 'Seven seas (open ocean)', 'Antarctica'], dtype=object)

In [5]:
world['name'].unique()

array(['Fiji', 'Tanzania', 'W. Sahara', 'Canada',
       'United States of America', 'Kazakhstan', 'Uzbekistan',
       'Papua New Guinea', 'Indonesia', 'Argentina', 'Chile',
       'Dem. Rep. Congo', 'Somalia', 'Kenya', 'Sudan', 'Chad', 'Haiti',
       'Dominican Rep.', 'Russia', 'Bahamas', 'Falkland Is.', 'Norway',
       'Greenland', 'Fr. S. Antarctic Lands', 'Timor-Leste',
       'South Africa', 'Lesotho', 'Mexico', 'Uruguay', 'Brazil',
       'Bolivia', 'Peru', 'Colombia', 'Panama', 'Costa Rica', 'Nicaragua',
       'Honduras', 'El Salvador', 'Guatemala', 'Belize', 'Venezuela',
       'Guyana', 'Suriname', 'France', 'Ecuador', 'Puerto Rico',
       'Jamaica', 'Cuba', 'Zimbabwe', 'Botswana', 'Namibia', 'Senegal',
       'Mali', 'Mauritania', 'Benin', 'Niger', 'Nigeria', 'Cameroon',
       'Togo', 'Ghana', "Côte d'Ivoire", 'Guinea', 'Guinea-Bissau',
       'Liberia', 'Sierra Leone', 'Burkina Faso', 'Central African Rep.',
       'Congo', 'Gabon', 'Eq. Guinea', 'Zambia', 'Malawi', 'M

In [2]:
import folium
map = folium.Map(location=[13.406, 80.110], tiles="CartoDB Positron", zoom_start=2)
map